In [7]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Acme_Synthetic_Data_final.csv')

# Define a function to calculate maximum possible sales
def calculate_max_sales(dataframe, growth_target):
    # Calculate potential max sales for each segment based on max trend
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    
    # Sum up the potential max sales to get a total
    total_max_sales = dataframe['Potential_Max_Sales'].sum()
    
    # Compare the maximum achievable growth with the target
    if total_max_sales < growth_target:
        shortfall = growth_target - total_max_sales
        message = f"Maximum Achievable Growth: ${total_max_sales / 1e6:.2f}M\n" \
                  f"Shortfall: ${shortfall / 1e6:.2f}M\n" \
                  f"The target growth of ${growth_target / 1e6:.2f}M is not achievable under current constraints."
    else:
        message = f"Maximum Achievable Growth: ${total_max_sales / 1e6:.2f}M\n" \
                  "The target growth is achievable."
    
    print(message)
    
    # Return the total and the detailed dataframe with the new column
    return dataframe

# Prompt the user for the growth target
growth_target_input = input("Please enter the growth target (in dollars): ")
try:
    growth_target = float(growth_target_input)  # Convert the input to a float
except ValueError:
    print("Invalid input! Please enter a numerical value.")
    exit()

# Execute the calculation
detailed_report = calculate_max_sales(df, growth_target)

# Optionally, display or save the detailed report showing sales, margin, max/min trend, max/min contribution
print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Potential_Max_Sales', 'Max_Trend', 'Min_Trend', 'Max_Contribution', 'Min_Contribution']].head())

detailed_report.to_csv('detailed_sales_maximization_report.csv', index=False)


Maximum Achievable Growth: $3.26M
Shortfall: $79999996.74M
The target growth of $80000000.00M is not achievable under current constraints.
   Portfolio      Geography Category    Brand    Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   7023   
1  Hair/Body  North America     Hair  Brand A    Shampoo   7601   
2  Hair/Body  North America     Hair  Brand B    Shampoo   5792   
3  Hair/Body  North America     Hair  Brand B  Body Wash   9678   
4  Hair/Body  North America     Hair  Brand C      Cream   1112   

   Potential_Max_Sales  Max_Trend  Min_Trend  Max_Contribution  \
0          9050.230455   0.288656  -0.044468          0.475025   
1          8508.344255   0.119372  -0.073998          0.294987   
2          7142.734685   0.233207  -0.010772          0.468395   
3         11764.618969   0.215604  -0.034484          0.249354   
4          1370.796987   0.232731  -0.128963          0.203140   

   Min_Contribution  
0          0.071955  
1          0.0873

In [8]:
def calculate_max_margin(dataframe):
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    
    dataframe['Potential_Max_Margin'] = dataframe['Potential_Max_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    total_max_margin = dataframe['Potential_Max_Margin'].sum()
    
    return total_max_margin, dataframe

total_max_margin, detailed_report = calculate_max_margin(df)

print(f"Maximum Achievable Margin: ${total_max_margin / 1e6:.2f}M")

print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Potential_Max_Sales', 'Potential_Max_Margin', 'Margin', 'Max_Trend', 'Max_Contribution']].head())

# If needed, save this detailed report to a CSV for further analysis or presentation
detailed_report.to_csv('detailed_margin_maximization_report2.csv', index=False)

Maximum Achievable Margin: $0.40M
   Portfolio      Geography Category    Brand    Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   7023   
1  Hair/Body  North America     Hair  Brand A    Shampoo   7601   
2  Hair/Body  North America     Hair  Brand B    Shampoo   5792   
3  Hair/Body  North America     Hair  Brand B  Body Wash   9678   
4  Hair/Body  North America     Hair  Brand C      Cream   1112   

   Potential_Max_Sales  Potential_Max_Margin  Margin  Max_Trend  \
0          9050.230455           2493.469443      58   0.288656   
1          8508.344255           1380.417302      55   0.119372   
2          7142.734685            401.474402      12   0.233207   
3         11764.618969           1026.744143      35   0.215604   
4          1370.796987             58.477451      21   0.232731   

   Max_Contribution  
0          0.475025  
1          0.294987  
2          0.468395  
3          0.249354  
4          0.203140  


In [15]:
def calculate_max_margin(dataframe, margin_target):
    # Calculate potential max sales for each segment based on max trend
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    
    # Assuming Margin is a percentage of Sales, we calculate the potential maximum margin
    # Margin potential is influenced by Max Contribution, which might affect pricing strategies
    dataframe['Potential_Max_Margin'] = dataframe['Potential_Max_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Sum up the potential max margin to get a total
    total_max_margin = dataframe['Potential_Max_Margin'].sum()
    
    # Compare the maximum achievable margin with the target
    if total_max_margin < margin_target:
        shortfall = margin_target - total_max_margin
        message = f"Maximum Achievable Margin: ${total_max_margin / 1e6:.2f}M\n" \
                  f"Shortfall: ${shortfall / 1e6:.2f}M\n" \
                  f"The target margin of ${margin_target / 1e6:.2f}M is not achievable under current constraints."
    else:
        message = f"Maximum Achievable Margin: ${total_max_margin / 1e6:.2f}M\n" \
                  "The target margin is achievable."
    
    print(message)
    
    # Return the detailed dataframe with the new columns
    return dataframe

# Prompt the user for the margin target
margin_target_input = input("Please enter the margin target (in dollars): ")
try:
    margin_target = float(margin_target_input)  # Convert the input to a float
except ValueError:
    print("Invalid input! Please enter a numerical value.")
    exit()

# Execute the calculation
detailed_report = calculate_max_margin(df, margin_target)

# Optionally, display or save the detailed report showing sales, margin, trend, and contribution for each unit
print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Potential_Max_Sales', 'Potential_Max_Margin', 'Margin', 'Max_Trend', 'Max_Contribution']].head())

# If needed, save this detailed report to a CSV for further analysis or presentation
detailed_report.to_csv('detailed_margin_maximization_report1.csv', index=False)

Maximum Achievable Margin: $0.40M
The target margin is achievable.
   Portfolio      Geography Category    Brand    Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   7023   
1  Hair/Body  North America     Hair  Brand A    Shampoo   7601   
2  Hair/Body  North America     Hair  Brand B    Shampoo   5792   
3  Hair/Body  North America     Hair  Brand B  Body Wash   9678   
4  Hair/Body  North America     Hair  Brand C      Cream   1112   

   Potential_Max_Sales  Potential_Max_Margin  Margin  Max_Trend  \
0          9050.230455           2493.469443      58   0.288656   
1          8508.344255           1380.417302      55   0.119372   
2          7142.734685            401.474402      12   0.233207   
3         11764.618969           1026.744143      35   0.215604   
4          1370.796987             58.477451      21   0.232731   

   Max_Contribution  
0          0.475025  
1          0.294987  
2          0.468395  
3          0.249354  
4          0.2031

In [19]:
def hit_sales_target_maximize_margin(dataframe, sales_target):
    # Assume that we can adjust sales up to Max_Trend and use Max_Contribution for margin calculations
    dataframe['Adjusted_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    dataframe['Potential_Margin'] = dataframe['Adjusted_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Initialize total sales and margin
    total_sales = dataframe['Adjusted_Sales'].sum()
    total_margin = dataframe['Potential_Margin'].sum()
    
    # If the total adjusted sales is less than the target, adjust by minimal necessary increments
    if total_sales < sales_target:
        needed_increase = sales_target - total_sales
        dataframe['Additional_Sales'] = (needed_increase / len(dataframe)) * dataframe['Max_Trend']
        dataframe['Adjusted_Sales'] += dataframe['Additional_Sales']
        dataframe['Potential_Margin'] = dataframe['Adjusted_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Recalculate totals
    total_sales = dataframe['Adjusted_Sales'].sum()
    total_margin = dataframe['Potential_Margin'].sum()
    
    return total_sales, total_margin, dataframe

# Define the sales target
sales_target = 50000000  # $50M

# Execute the function
total_sales, total_margin, detailed_report = hit_sales_target_maximize_margin(df, sales_target)

# Output results
print(f"Total Adjusted Sales to Meet Target: ${total_sales:.2f}")
print(f"Total Potential Margin from Adjusted Sales: ${total_margin:.2f}")

# Display detailed results for each unit
print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Adjusted_Sales', 'Potential_Margin', 'Max_Trend', 'Max_Contribution']].head())

# Optionally, save this detailed report to a CSV for further analysis or presentation
detailed_report.to_csv('detailed_sales_margin_optimization_report3.csv', index=False)

Total Adjusted Sales to Meet Target: $12708955.59
Total Potential Margin from Adjusted Sales: $1569048.21
   Portfolio      Geography Category    Brand    Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   7023   
1  Hair/Body  North America     Hair  Brand A    Shampoo   7601   
2  Hair/Body  North America     Hair  Brand B    Shampoo   5792   
3  Hair/Body  North America     Hair  Brand B  Body Wash   9678   
4  Hair/Body  North America     Hair  Brand C      Cream   1112   

   Adjusted_Sales  Potential_Margin  Max_Trend  Max_Contribution  
0    36036.339673       9928.533008   0.288656          0.475025  
1    19668.266829       3191.034003   0.119372          0.294987  
2    28944.984873       1626.921763   0.233207          0.468395  
3    31921.222499       2785.889482   0.215604          0.249354  
4    23128.560359        986.651757   0.232731          0.203140  


In [ ]:
import pandas as pd

def hit_margin_target_maximize_sales(dataframe, margin_target):
    # First, calculate potential max sales and margins at their maximum allowed trend and contribution
    dataframe['Potential_Max_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
    dataframe['Potential_Margin_Amount'] = dataframe['Potential_Max_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
    
    # Calculate initial total potential margin
    total_potential_margin = dataframe['Potential_Margin_Amount'].sum()
    
    # If the total potential margin is less than the target, adjust by increasing sales within max constraints
    if total_potential_margin < margin_target:
        # Increase sales while trying to maintain or increase margin percentage
        for index, row in dataframe.iterrows():
            if row['Potential_Margin_Amount'] < (row['Margin'] / 100) * margin_target:
                # Increase sales to attempt to meet part of the margin shortfall
                additional_sales_needed = (margin_target - total_potential_margin) / len(dataframe)
                adjusted_sales = min(row['Potential_Max_Sales'] + additional_sales_needed, row['Sales'] * (1 + row['Max_Trend']))
                dataframe.at[index, 'Adjusted_Sales'] = adjusted_sales
                dataframe.at[index, 'Potential_Margin_Amount'] = adjusted_sales * (row['Margin'] / 100) * row['Max_Contribution']
        
        # Recalculate total potential margin
        total_potential_margin = dataframe['Potential_Margin_Amount'].sum()

    return total_potential_margin, dataframe

# Define the margin target
margin_target = float(input("Please enter the margin target (in dollars): "))

# Execute the function
total_potential_margin, detailed_report = hit_margin_target_maximize_sales(df, margin_target)

# Output results
print(f"Total Potential Margin from Adjusted Sales: ${total_potential_margin:.2f}")

# Display detailed results for each unit
print(detailed_report[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', 'Sales', 'Adjusted_Sales', 'Potential_Margin_Amount', 'Max_Trend', 'Max_Contribution']].head())

detailed_report.to_csv('detailed_margin_sales_optimization_report.csv', index=False)


Total Potential Margin from Adjusted Sales: $404187.19
   Portfolio      Geography Category    Brand    Segment  Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   7023   
1  Hair/Body  North America     Hair  Brand A    Shampoo   7601   
2  Hair/Body  North America     Hair  Brand B    Shampoo   5792   
3  Hair/Body  North America     Hair  Brand B  Body Wash   9678   
4  Hair/Body  North America     Hair  Brand C      Cream   1112   

   Adjusted_Sales  Potential_Margin_Amount  Max_Trend  Max_Contribution  
0     9050.230455              2493.469443   0.288656          0.475025  
1     8508.344255              1380.417302   0.119372          0.294987  
2     7142.734685               401.474402   0.233207          0.468395  
3    11764.618969              1026.744143   0.215604          0.249354  
4     1370.796987                58.477451   0.232731          0.203140  


In [22]:
import numpy as np
def project_annual_data(dataframe):
    # Create a dictionary to store yearly data
    projections = {}
    
    # Initial settings for the projections
    for year in range(1, 6):  # For each of the next 5 years
        if year == 1:
            # For the first year, use existing data and max trend to project next year's figures
            dataframe[f'Year_{year}_Sales'] = dataframe['Sales'] * (1 + dataframe['Max_Trend'])
            dataframe[f'Year_{year}_Margin'] = dataframe[f'Year_{year}_Sales'] * (dataframe['Margin'] / 100) * dataframe['Max_Contribution']
        else:
            # For subsequent years, use the previous year's data to project further, allowing user to modify growth trends annually
            growth_trend = np.random.uniform(dataframe['Min_Trend'], dataframe['Max_Trend'])  # Simulate annual adjustment of trends
            contribution_adjustment = np.random.uniform(dataframe['Min_Contribution'], dataframe['Max_Contribution'])  # Simulate annual adjustment of contribution
            
            dataframe[f'Year_{year}_Sales'] = dataframe[f'Year_{year - 1}_Sales'] * (1 + growth_trend)
            dataframe[f'Year_{year}_Margin'] = dataframe[f'Year_{year}_Sales'] * (dataframe['Margin'] / 100) * contribution_adjustment
        
        # Store each year's data in the dictionary
        projections[f'Year_{year}'] = dataframe[['Portfolio', 'Geography', 'Category', 'Brand', 'Segment', f'Year_{year}_Sales', f'Year_{year}_Margin']]
    
    return projections

# Execute the function
annual_projections = project_annual_data(df)

# Print results for each year
for year in annual_projections:
    print(f"Data for {year}:")
    print(annual_projections[year].head(), "\n")  # Displaying only the first few rows for brevity
    annual_projections[year].to_csv(f'{year}_sales_margin_projection.csv', index=False)


Data for Year_1:
   Portfolio      Geography Category    Brand    Segment  Year_1_Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   9050.230455   
1  Hair/Body  North America     Hair  Brand A    Shampoo   8508.344255   
2  Hair/Body  North America     Hair  Brand B    Shampoo   7142.734685   
3  Hair/Body  North America     Hair  Brand B  Body Wash  11764.618969   
4  Hair/Body  North America     Hair  Brand C      Cream   1370.796987   

   Year_1_Margin  
0    2493.469443  
1    1380.417302  
2     401.474402  
3    1026.744143  
4      58.477451   

Data for Year_2:
   Portfolio      Geography Category    Brand    Segment  Year_2_Sales  \
0  Hair/Body  North America     Hair  Brand A      Cream   8704.100274   
1  Hair/Body  North America     Hair  Brand A    Shampoo   8547.416722   
2  Hair/Body  North America     Hair  Brand B    Shampoo   8781.993863   
3  Hair/Body  North America     Hair  Brand B  Body Wash  13700.545797   
4  Hair/Body  North America     Hai